<a href="https://colab.research.google.com/github/nssn96/ML_Neural-networks/blob/main/ML_nn_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Assignment 2 - Neural Networks


1.   Author : Surya Narayanan Nadhamuni Suresh
2.   UTA ID : 1001877873


In [12]:
#import lines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
#References Used
#https://towardsdatascience.com/https-medium-com-piotr-skalski92-deep-dive-into-deep-networks-math-17660bc376ba
#https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795
#https://towardsdatascience.com/an-introduction-to-neural-networks-with-implementation-from-scratch-using-python-da4b6a45c05b

# This is the Layer class

class Layer:

  def initial_w_bias(self,nn):
    param = {}
    #creating the values for the w and bias matrix
    for i in range(1,len(nn)):
      param['w' + str(i)] = np.random.randn(nn[i], nn[i-1])*0.01
      param['B' + str(i)] = np.random.randn(nn[i],1)*0.01
    return param


In [14]:
#This is the class for the hyperbolic tangent functions

#class htangent_activation(Layer):

def tanh_forwardprop(z):
  #2/(1 + e-2x) - 1
  return (2/(1+np.exp(-2*z)) -1)

def tanh_backprop(dA,z):
  value = tanh_forwardprop(z)
  return dA * value * (1-value)


In [15]:
#This is the class for the sigmoid function

#class sigmoid_activation(Layer):

def sigmoid_forwardprop(Z):
  return 1/(1+np.exp(-Z))
  
def sigmoid_backprop(dA,z):
  value = sigmoid_forwardprop(z)
  return dA * value * (1-value)

In [16]:
#This is the class for the softmax function

class softmax_activation(Layer):

  def softmax_forwardprop(self,z):
    return np.exp(z) / sum(np.exp(z))
  

  #derivative of softmax
  def softmax_backprop(self,z):
    exp=np.exp(z-z.max())
    return exp/np.sum(exp,axis=0)*(1-exp/np.sum(exp,axis=0))
    



  
  #Yet to edit this--- DONT FORGET+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  # def softmax_backprop(probs, bp_err):
  #   dim = probs.shape[1]
  #   output = np.empty(probs.shape)
  #   for j in range(dim):
  #       d_prob_over_xj = - (probs * probs[:,[j]])  # i.e. prob_k * prob_j, no matter k==j or not
  #       d_prob_over_xj[:,j] += probs[:,j]   # i.e. when k==j, +prob_j
  #       output[:,j] = np.sum(bp_err * d_prob_over_xj, axis=1)
  #   return output
  

In [17]:
# This is the Linear layer class

class LinearLayer(Layer):


  def forward(self,data,param):
    
    #Z_mat = np.dot(w,prev_out)
    
    layer_num = len(param)//2
    backup = {} # to store info required for the backward calculation

    #traversing over the layers in the neural network
    #this is the same steps that we used to assign the weights and bias for network
    for i in range(1,layer_num+1):
      #if it is the first iteration the input will be the input ie the train or the test data
      w = param['w'+str(i)]
      bias = param['b'+str(i)]
      if i==1:
        #Z_mat--> the input matrix calculation which will be passed in the activation function
        Z_mat = np.dot(w, data) + bias
        backup['A' + str(i)] = tanh_forwardprop(Z_mat)
      else:
        Z_mat = np.dot(w, backup['A' + str(i-1)]) + bias
        if i==layer_num:
          backup['A' + str(i)] = Z_mat
        else:
          backup['A' + str(i)] = tanh_forwardprop(Z_mat)

      #storing the values for backward pass call
      backup['Z' + str(i)] = Z_mat

    return backup
  
  def compute_loss(self,backup,target):
    layer_num = len(backup)//2
    y_pred = backup['A' + str(layer_num)]
    loss = cost = 1/(2*len(target)) * np.sum(np.square(y_pred - target))
    return loss
  
  def backward(self,data,target,param,backup):
    deriv_values= {}  #dictionary to store the derivative values
    layer_num = len(target)//2
    n = len(target)
     #since we need to go backward in backward propagation
    for i in range(layer_num,0,-1):

      A_previous = backup['A'+str(i)]
      z_current = backup['Z'+str(i)]
      w = param['w'+str(i)]
      bias = param['b'+str(i)]

      if i == layer_num:
        dA_current = 1/n * (A_previous - target)
        dz = dA_current
      else:
        dA_current = np.dot(param['w' + str(i+1)].T, dz)
        dz = np.multiply(dA_current, np.where(A_previous>=0, 1, 0))
      if i==1:
        dw = 1/n * np.dot(dz, data.T)
        dbias = 1/n * np.sum(dz, axis=1, keepdims=True)
      else:
        dw = 1/n * np.dot(dz,backup['A' + str(i-1)].T)
        dbias = 1/n * np.sum(dz, axis=1, keepdims=True)
      
      deriv_values['dw' + str(i)] = dw
      deriv_values['db' + str(i)] = dbias

    return deriv_values

      














